## 1. Scrape IRS data

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime
import pandas as pd
import sys 

# Open the website using Selenium WebDriver
path = '/Users/youmjaehwa/Downloads/chromedriver_mac_arm64/chromedriver'
sys.path.append(path)
driver = webdriver.Chrome()

url = 'http://www.smbs.biz/Exchange/IRS.jsp'
driver.get(url)
html = driver.page_source

# Parse the HTML content using Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')
table = soup.find('table', summary='기일물/1Y/2Y/3Y/4Y/5Y/7Y/10Y')

# Extract the column names
columns = [th.text.strip() for th in table.select('thead th')]

# Extract the search date
search_date_value = driver.find_element(By.ID,"searchDate").get_attribute('value')
search_date = datetime.strptime(search_date_value, "%Y.%m.%d").date()
# print(search_date)

# Extract the data from the table and store it in a list of lists
data = []
for row in table.find_all('tr')[1:]: # exclude first row NaN
    cells = row.find_all('td')
    row_data = [cell.text.strip() for cell in cells]
    data.append(row_data)        

# Convert the list of lists into a Pandas DataFrame
df = pd.DataFrame(data, columns=columns)

# Display the dataframe
print(df)

# Close the browser
driver.quit()

     기일물 Receive     Mid     Pay
0     6M  3.8100  3.8050  3.8000
1     9M  3.8250  3.8200  3.8150
2     1Y  3.8050  3.8000  3.7950
3   1.5Y  3.7375  3.7325  3.7275
4     2Y  3.6650  3.6600  3.6550
5     3Y  3.5650  3.5600  3.5550
6     4Y  3.5050  3.5000  3.4950
7     5Y  3.4650  3.4600  3.4550
8     6Y  3.4400  3.4350  3.4300
9     7Y  3.4225  3.4175  3.4125
10    8Y  3.4175  3.4125  3.4075
11    9Y  3.4150  3.4100  3.4050
12   10Y  3.4125  3.4075  3.4025
13   12Y  3.4200  3.4150  3.4100
14   15Y  3.4150  3.4100  3.4050
15   20Y  3.3650  3.3600  3.3550


## 2. Set Table

### 1) Function for next business day

In [2]:
import datetime
import holidays

ONE_DAY = datetime.timedelta(days=1)
HOLIDAYS_US = holidays.US()

def next_business_day(date_str):
    date = datetime.datetime.strptime(date_str, "%Y-%m-%d").date()
    next_day = date + ONE_DAY
    while next_day.weekday() in holidays.WEEKEND or next_day in HOLIDAYS_US:
        next_day += ONE_DAY
    return next_day

# print(next_business_day('2024-03-16')), returns '2024-03-18'

### 2) Set table format

In [3]:
import pandas as pd
from datetime import date
from dateutil.relativedelta import relativedelta
import numpy as np

# Define the column names
columns = ["tenor", "maturity", "adjusted maturity", "market quote"]

# Create an empty DataFrame
df = pd.DataFrame(columns=columns)

# Set the "tenor" column as the index
df.set_index("tenor", inplace=True)

# Define the range of tenor values
tenor_range = np.arange(0.25, 3.25, 0.25)

# Calculate the maturity and adjusted maturity for each tenor
for tenor in tenor_range:
    maturity = search_date + relativedelta(months=tenor*12)
    adjusted_maturity = next_business_day(str(maturity))  # Assuming next business day calculation
    
    # Add the row to the DataFrame
    df.loc[tenor] = [maturity, adjusted_maturity, np.nan]

# Perform the bootstrapping calculation to fill the "market quote" column
# Replace the np.nan values with the actual market quotes

# Display the resulting DataFrame
print(df)

         maturity adjusted maturity  market quote
tenor                                            
0.25   2023-09-16        2023-09-18           NaN
0.50   2023-12-16        2023-12-18           NaN
0.75   2024-03-16        2024-03-18           NaN
1.00   2024-06-16        2024-06-17           NaN
1.25   2024-09-16        2024-09-17           NaN
1.50   2024-12-16        2024-12-17           NaN
1.75   2025-03-16        2025-03-17           NaN
2.00   2025-06-16        2025-06-17           NaN
2.25   2025-09-16        2025-09-17           NaN
2.50   2025-12-16        2025-12-17           NaN
2.75   2026-03-16        2026-03-17           NaN
3.00   2026-06-16        2026-06-17           NaN
